In [1]:
import tensorflow as tf

In [6]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [3]:
dy_dw = t.gradient(y, w)

In [4]:
dy_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [7]:
dz_dw = t.gradient(z, w)

In [8]:
dz_dw

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

加载mnist数据集

In [2]:
(train_image, train_label), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

In [10]:
train_image.shape

(60000, 28, 28)

In [3]:
# 扩展维度
train_image = tf.expand_dims(train_image, -1)

In [4]:
test_image = tf.expand_dims(test_image, -1)

In [12]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [5]:
# 改变数据类型
train_image = tf.cast(train_image/255, tf.float32)

In [6]:
test_image = tf.cast(test_image/255, tf.float32)

In [7]:
train_label = tf.cast(train_label, tf.int64)

In [8]:
test_label = tf.cast(test_labels, tf.int64)

In [15]:
train_label

<tf.Tensor: shape=(60000,), dtype=int64, numpy=array([5, 0, 4, ..., 5, 6, 8], dtype=int64)>

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))

In [10]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_label))

In [17]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [11]:
# 乱序
dataset = dataset.shuffle(10000).batch(32)

In [12]:
test_dataset = test_dataset.batch(32)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [14]:
optimizer = tf.keras.optimizers.Adam()

In [15]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
features, labels = next(iter(dataset))

In [24]:
features.shape

TensorShape([32, 28, 28, 1])

In [25]:
labels.shape

TensorShape([32])

In [26]:
predictions = model(features)

In [27]:
predictions.shape

TensorShape([32, 10])

In [28]:
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)>

In [29]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 0, 0, 6, 9, 3, 7, 1, 0, 3, 9, 9, 3, 6, 6, 5, 4, 1, 8, 2, 9, 5,
       8, 8, 1, 8, 3, 0, 4, 1, 7, 6], dtype=int64)>

In [16]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [20]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [18]:
# 
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    # grads 梯度
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [21]:
# 
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [23]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch {} loss is {}, accuracy is {}'.format(epoch,
                                                           train_loss.result(),
                                                           train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                                     test_loss.result(),
                                                                     test_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [24]:
train()

Epoch 0 loss is 0.1988174170255661, accuracy is 0.9365833401679993
Epoch 0 test_loss is 0.1920831948518753, test_accuracy is 0.9373000264167786
Epoch 1 loss is 0.19330810010433197, accuracy is 0.9386500120162964
Epoch 1 test_loss is 0.18655160069465637, test_accuracy is 0.9379000067710876


KeyboardInterrupt: 

### tf.keras.metrics 汇总计算模块

In [40]:
m = tf.keras.metrics.Mean('acc')

In [41]:
m(19)

<tf.Tensor: shape=(), dtype=float32, numpy=19.0>

In [42]:
m(32)

<tf.Tensor: shape=(), dtype=float32, numpy=25.5>

In [43]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=25.5>

In [44]:
# 重置
m.reset_states()

In [45]:
m(5)

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [46]:
m(15)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [47]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [53]:
a.reset_states()

In [54]:
a(labels, predictions)

<tf.Tensor: shape=(), dtype=float32, numpy=0.125>

In [55]:
4/32

0.125